In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

### Functions

In [2]:
def scrape_table(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.content,'lxml')
    table = soup.find_all('table')[0] 
    table_rows = table.find_all('tr')

    x = []
    for tr in table_rows:
        td = tr.find_all('td')
        row = [tr.text for tr in td]
        x.append(row)

    df = pd.DataFrame(x)
    return df

In [3]:
def clean_table(df):
    headers = df.iloc[1]
    df.columns = headers
    df.rename(columns={df.columns[0]: "country"}, inplace=True)
    df = df[2:]
    df.set_index('country', inplace=True)
    return df

In [4]:
def remove_nan(df):
    df = df.replace(r'^\s*$', np.nan, regex=True)
    df = df.dropna(axis='rows', how='any')
    df = df.apply(pd.to_numeric)
    return df

In [5]:
def open_csv(table):
    table = pd.read_csv(table, error_bad_lines=False, encoding='latin-1')
    table.columns = table.columns.str.strip()
    table = table.drop('Country Code', 1)
    table.columns = table.columns.str.split(" ").str[0]
    table.drop(['Series'], axis=1, inplace=True)
    return table

In [6]:
def clean_csv(df):
    df.set_index('Country', inplace=True)
    df.replace('..',np.nan, inplace=True)
    df = df.apply(pd.to_numeric)
    return df

In [7]:
def year_filter(table):
    table_filter = table.loc[:, '2000': '2015']
    return table_filter

### Potential Support Ratio (PSR) [working population/population over 60]

In [8]:
workpop = open_csv('workpop.csv')

In [10]:
workpop = clean_csv(workpop)

In [12]:
workpop = year_filter(workpop)

In [13]:
pop60 = open_csv('pop_60.csv')

In [14]:
pop60 = clean_csv(pop60)

In [16]:
pop60 = year_filter(pop60)

### Generating PSR (Y)

In [17]:
y = workpop / pop60
y = y.dropna()
y.shape

(193, 16)

In [18]:
y.head()

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,
Afghanistan,45.955507,46.201617,46.186552,45.998473,45.683139,45.218910,44.996441,44.864143,44.680468,44.280740,43.592701,42.239796,40.597162,38.839132,37.152189,35.605193
Albania,8.422533,7.988096,7.548263,7.113599,6.700764,6.315733,5.893386,5.523944,5.195871,4.903550,4.644120,4.351654,4.100328,3.880106,3.683892,3.511022
Algeria,14.506410,13.460978,12.549468,11.748408,11.044890,10.439179,9.953409,9.568926,9.270336,9.044022,8.878232,8.902922,8.938170,8.978466,9.001591,8.982427
Angola,40.942691,40.982135,41.015405,41.074604,41.179485,41.331717,41.570159,41.749992,41.920427,42.138134,42.406579,42.721178,43.001070,43.124449,42.917746,42.308292
Antigua and Barbuda,8.585382,8.571939,8.492035,8.361447,8.219993,8.068043,7.945557,7.813955,7.688295,7.573103,7.476652,7.362556,7.263806,7.167139,7.037576,6.870103


### Fertility (X1)

In [19]:
url = 'https://docs.google.com/spreadsheets/d/1oq3r8W7ajenKFgoAYoOf2MXeTWWNPpudR-Fo5m2-o30/pub'
fertility = scrape_table(url)

In [20]:
fertility = clean_table(fertility)

In [21]:
x1 = year_filter(fertility)
x1 = remove_nan(x1)
x1.head()

1,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
country,,,,,,,,,,,,,,,,
Afghanistan,7.73,7.62,7.48,7.32,7.14,6.93,6.70,6.46,6.20,5.93,5.66,5.40,5.14,4.90,4.68,4.47
Albania,2.38,2.29,2.20,2.10,2.00,1.92,1.85,1.80,1.76,1.74,1.74,1.75,1.76,1.77,1.78,1.78
Algeria,2.51,2.44,2.41,2.41,2.45,2.51,2.58,2.66,2.73,2.78,2.82,2.83,2.82,2.80,2.76,2.71
Angola,6.84,6.81,6.78,6.74,6.70,6.66,6.60,6.52,6.43,6.33,6.22,6.10,5.98,5.86,5.75,5.65
Antigua and Barbuda,2.32,2.31,2.29,2.27,2.25,2.22,2.20,2.18,2.16,2.15,2.13,2.12,2.10,2.09,2.08,2.06


### Birth Rate (X2)

In [22]:
url = 'https://docs.google.com/spreadsheets/d/1QkK8B3EnGoWzcHUmdf0AIU8YHk5LmzbOcsRRKbN9w2Y/pub'
birth_rate = scrape_table(url)

In [23]:
birth_rate = clean_table(birth_rate)

In [24]:
x2 = year_filter(birth_rate)
x2 = remove_nan(x1)
x2.head()

1,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
country,,,,,,,,,,,,,,,,
Afghanistan,7.73,7.62,7.48,7.32,7.14,6.93,6.70,6.46,6.20,5.93,5.66,5.40,5.14,4.90,4.68,4.47
Albania,2.38,2.29,2.20,2.10,2.00,1.92,1.85,1.80,1.76,1.74,1.74,1.75,1.76,1.77,1.78,1.78
Algeria,2.51,2.44,2.41,2.41,2.45,2.51,2.58,2.66,2.73,2.78,2.82,2.83,2.82,2.80,2.76,2.71
Angola,6.84,6.81,6.78,6.74,6.70,6.66,6.60,6.52,6.43,6.33,6.22,6.10,5.98,5.86,5.75,5.65
Antigua and Barbuda,2.32,2.31,2.29,2.27,2.25,2.22,2.20,2.18,2.16,2.15,2.13,2.12,2.10,2.09,2.08,2.06


### Death Rate (X3)

In [25]:
death_rate = open_csv('death_rate.csv')

In [26]:
death_rate = clean_csv(death_rate)

In [27]:
death_rate.shape

(217, 58)

In [28]:
x3 = year_filter(death_rate)
x3 = x3.dropna()
x3.head()

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,
Afghanistan,11.959,11.574,11.167,10.748,10.325,9.906,9.498,9.108,8.742,8.403,8.095,7.816,7.563,7.331,7.118,6.924
Albania,5.925,5.892,5.907,5.973,6.086,6.231,6.386,6.527,6.640,6.720,6.776,6.822,6.879,6.963,7.076,7.217
Algeria,5.072,4.988,4.917,4.857,4.809,4.773,4.749,4.738,4.737,4.744,4.754,4.766,4.774,4.779,4.778,4.773
Angola,17.858,17.029,16.189,15.357,14.548,13.773,13.034,12.327,11.654,11.023,10.450,9.949,9.525,9.176,8.899,8.683
Antigua and Barbuda,6.443,6.353,6.273,6.206,6.154,6.114,6.082,6.052,6.019,5.982,5.941,5.900,5.862,5.831,5.808,5.794


### Life Expectancy (X4)

In [29]:
url = 'https://docs.google.com/spreadsheets/d/1H3nzTwbn8z4lJ5gJ_WfDgCeGEXK3PVGcNjQ_U5og8eo/pub'
life_expec = scrape_table(url)

In [30]:
life_expec = clean_table(life_expec)

In [31]:
x4 = year_filter(life_expec)
x4 = remove_nan(x4)
x4.head()

1,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
country,,,,,,,,,,,,,,,,
Afghanistan,50.1,50.4,51.0,51.4,51.8,52.0,52.1,52.4,52.8,53.3,53.6,54.0,54.4,54.8,54.9,53.8
Albania,74.7,75.1,75.5,75.7,75.9,76.2,76.4,76.6,76.8,77.0,77.2,77.4,77.5,77.7,77.9,78.0
Algeria,73.3,73.5,73.8,73.9,74.4,74.8,75.0,75.3,75.5,75.7,76.0,76.1,76.2,76.3,76.3,76.4
American Samoa,71.7,71.7,71.7,71.7,71.8,72.0,72.2,72.4,72.7,71.3,72.8,72.6,72.6,72.7,72.8,72.9
Andorra,83.7,83.9,84.1,84.2,84.1,84.3,84.4,84.5,84.6,84.6,84.7,84.7,84.7,84.8,84.8,84.8


### GDP Data Scrape (X5)

In [32]:
gdp = open_csv('gdp.csv')

In [33]:
gdp = clean_csv(gdp)

In [34]:
gdp.shape

(217, 58)

In [35]:
x5 = year_filter(gdp)
x5 = x5.dropna()
x5.head()

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,
Albania,7.348639,8.958088,4.544406,6.169982,6.152274,6.263276,6.098292,6.703338,4.560118,4.051449,4.226219,2.826410,1.587602,1.185969,1.985308,2.516853
Algeria,2.427876,1.676476,4.279453,5.850521,2.943232,4.452752,0.210972,1.799197,0.713953,-0.101600,1.763682,0.930355,1.321109,0.696354,1.733281,1.790254
Andorra,1.913452,1.517845,2.354989,7.366657,3.324674,3.825118,1.794873,-2.007178,-9.874045,-4.375955,-5.344257,-3.851846,-0.039744,2.392514,4.298114,2.404977
Angola,-0.024041,0.878294,9.853363,0.931398,6.368053,16.659902,14.820467,18.875797,9.832697,-1.175597,-0.175344,0.230112,1.499648,3.135079,1.117471,-0.471272
Antigua and Barbuda,4.454106,-6.600061,-0.391731,4.810140,4.581538,5.224918,11.420222,7.968122,-1.216004,-13.143488,-8.258792,-3.161465,2.375039,-1.173116,3.984751,2.968827


### Creating Dictionary for Final Dataframe

In [36]:
features = { 'psr': y, 'fertility': x1, 'birth_rate': x2, 'death_rate': x3, 'life_expectancy': x4, 'gdp': x5}
results = {}
for variable in features:
    table = features[variable]
    years = list(table.columns)
    countries = list(table.index)
    for year in years:
        for country in countries:
                key = (country, year)
                if  variable in results:
                    results[variable][key] = table[year][country]
                else:
                    results[variable] = {}
                    results[variable][key] = table[year][country]

### Generating Concatenated Dataframe

In [37]:
psr_data = pd.DataFrame(results)

In [38]:
psr_data
psr_data = psr_data.dropna()

In [82]:
pd.options.display.max_rows = 30
psr_data.groupby(level=[0]).size()

Albania                 16
Algeria                 16
Angola                  16
Antigua and Barbuda     16
Argentina               16
Armenia                 16
Australia               16
Austria                 16
Azerbaijan              16
Bahrain                 16
Bangladesh              16
Barbados                16
Belarus                 16
Belgium                 16
Belize                  16
                        ..
Tunisia                 16
Turkey                  16
Turkmenistan            16
Uganda                  16
Ukraine                 16
United Arab Emirates    16
United Kingdom          16
United States           16
Uruguay                 16
Uzbekistan              16
Vanuatu                 16
Vietnam                 16
West Bank and Gaza      16
Zambia                  16
Zimbabwe                16
Length: 163, dtype: int64

In [45]:
psr_data.to_pickle('psr_data_2000.pkl')

In [46]:
psr_data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2608 entries, (Albania, 2000) to (Zimbabwe, 2015)
Data columns (total 6 columns):
psr                2608 non-null float64
fertility          2608 non-null float64
birth_rate         2608 non-null float64
death_rate         2608 non-null float64
life_expectancy    2608 non-null float64
gdp                2608 non-null float64
dtypes: float64(6)
memory usage: 131.9+ KB


In [47]:
psr_data.describe()

,psr,fertility,birth_rate,death_rate,life_expectancy,gdp
count,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000,2608.000000
mean,13.778995,2.967745,2.967745,8.593599,69.972086,2.554500
std,11.101960,1.545346,1.545346,3.343699,9.023160,5.445198
min,2.458244,1.130000,1.130000,1.473000,32.200000,-62.225087
25%,3.888695,1.770000,1.770000,6.133500,63.700000,0.451389
50%,10.306088,2.440000,2.440000,8.000000,72.400000,2.427944
75%,21.695212,3.990000,3.990000,10.290500,77.000000,4.623632
max,47.159055,7.730000,7.730000,23.986000,83.300000,122.968301


In [192]:
def test_filter(df):
    country_lst = ['Japan', 'Italy', 'Germany', 'Finland', 'Portugal', 'Sweden', 'Bulgaria', 'Greece',
                         'France', 'Denmark', 'India', 'Bolivia', 'Indonesia', 'Fiji', 'Guatemala', 'Cambodia',
                         'Nepal', 'South Africa', 'Paraguay', 'Samoa', 'Gabon', 'Angola', 'Sierra Leone',
                         'Equatorial Guinea', 'Burkina Faso', 'United Arab Emirates', 'Bahrain', 'Kuwait', 'Oman',
                         'Saudi Arabia', 'Rwanda', 'Burundi', 'Belgium']
    df_filter = df.loc[country_lst, :]
    return df_filter

In [199]:
psr_data_test = test_filter(psr_data)
psr_data_test.to_pickle('psr_data_test.pkl')
psr_data_test

psr  fertility  birth_rate  death_rate  \
Angola               2000  40.942691       6.84        6.84      17.858   
                     2001  40.982135       6.81        6.81      17.029   
                     2002  41.015405       6.78        6.78      16.189   
                     2003  41.074604       6.74        6.74      15.357   
                     2004  41.179485       6.70        6.70      14.548   
                     2005  41.331717       6.66        6.66      13.773   
                     2006  41.570159       6.60        6.60      13.034   
                     2007  41.749992       6.52        6.52      12.327   
                     2008  41.920427       6.43        6.43      11.654   
                     2009  42.138134       6.33        6.33      11.023   
                     2010  42.406579       6.22        6.22      10.450   
                     2011  42.721178       6.10        6.10       9.949   
                     2012  43.001070       5.98        5.98       9.525   
                     2013  43.124449       5.86        5.86       9.176   
                     2014  42.917746       5.75        5.75       8.899   
...                              ...        ...         ...         ...   
United Arab Emirates 2001  31.627997       2.53        2.53       1.852   
                     2002  29.794506       2.43        2.43       1.794   
                     2003  28.313812       2.33        2.33       1.738   
                     2004  27.089200       2.24        2.24       1.682   
                     2005  26.073650       2.15        2.15       1.629   
                     2006  25.225840       2.07        2.07       1.581   
                     2007  24.327325       2.00        2.00       1.540   
                     2008  23.561727       1.95        1.95       1.508   
                     2009  22.821341       1.90        1.90       1.485   
                     2010  21.922062       1.87        1.87       1.474   
                     2011  20.681146       1.84        1.84       1.473   
                     2012  19.330486       1.82        1.82       1.482   
                     2013  18.174879       1.80        1.80       1.499   
                     2014  17.407598       1.78        1.78       1.524   
                     2015  17.010631       1.77        1.77       1.555   

                           life_expectancy        gdp  
Angola               2000             52.3  -0.024041  
                     2001             52.5   0.878294  
                     2002             53.3   9.853363  
                     2003             53.9   0.931398  
                     2004             54.5   6.368053  
                     2005             55.2  16.659902  
                     2006             55.7  14.820467  
                     2007             56.2  18.875797  
                     2008             56.7   9.832697  
                     2009             57.1  -1.175597  
                     2010             57.6  -0.175344  
                     2011             58.1   0.230112  
                     2012             58.5   1.499648  
                     2013             58.8   3.135079  
                     2014             59.2   1.117471  
...                                    ...        ...  
United Arab Emirates 2001             73.6  -3.817369  
                     2002             73.8  -2.858734  
                     2003             74.1   1.976396  
                     2004             74.4   0.292827  
                     2005             75.2  -6.401402  
                     2006             75.7  -4.043543  
                     2007             75.6 -10.507962  
                     2008             75.6  -9.533910  
                     2009             75.6 -14.786310  
                     2010             75.6  -5.820719  
                     2011             75.5   1.976251  
                     2012             75.5   1.808336  
        

In [194]:
set1 = set(psr_data.index.get_level_values(0))
set2 = set(psr_data_test.index.get_level_values(0))

In [195]:
psr_data2 = list(set1.difference(set2))

In [196]:
def trainval_filter(df, country_lst):
    df_filter = df.loc[country_lst, :]
    return df_filter

In [197]:
psr_data_trainval = trainval_filter(psr_data, psr_data2)

In [200]:
psr_data_trainval.to_pickle('psr_data_trainval.pkl')
psr_data_trainval

psr  fertility  birth_rate  death_rate  life_expectancy  \
Albania  2000   8.422533       2.38        2.38       5.925             74.7   
         2001   7.988096       2.29        2.29       5.892             75.1   
         2002   7.548263       2.20        2.20       5.907             75.5   
         2003   7.113599       2.10        2.10       5.973             75.7   
         2004   6.700764       2.00        2.00       6.086             75.9   
         2005   6.315733       1.92        1.92       6.231             76.2   
         2006   5.893386       1.85        1.85       6.386             76.4   
         2007   5.523944       1.80        1.80       6.527             76.6   
         2008   5.195871       1.76        1.76       6.640             76.8   
         2009   4.903550       1.74        1.74       6.720             77.0   
         2010   4.644120       1.74        1.74       6.776             77.2   
         2011   4.351654       1.75        1.75       6.822             77.4   
         2012   4.100328       1.76        1.76       6.879             77.5   
         2013   3.880106       1.77        1.77       6.963             77.7   
         2014   3.683892       1.78        1.78       7.076             77.9   
...                  ...        ...         ...         ...              ...   
Zimbabwe 2001  26.775327       4.04        4.04      16.611             47.0   
         2002  26.513951       4.02        4.02      16.771             45.9   
         2003  26.344482       4.00        4.00      16.730             45.3   
         2004  26.253110       3.99        3.99      16.487             44.7   
         2005  26.246711       3.97        3.97      16.040             45.1   
         2006  26.031012       3.94        3.94      15.398             45.5   
         2007  25.896808       3.90        3.90      14.604             46.4   
         2008  25.879714       3.85        3.85      13.713             47.3   
         2009  26.033304       3.79        3.79      12.774             48.0   
         2010  26.381177       3.72        3.72      11.838             49.1   
         2011  26.666183       3.64        3.64      10.954             51.6   
         2012  27.084128       3.56        3.56      10.148             54.2   
         2013  27.554562       3.49        3.49       9.441             55.7   
         2014  27.944255       3.41        3.41       8.856             57.0   
         2015  28.160703       3.35        3.35       8.402             59.3   

                     gdp  
Albania  2000   7.348639  
         2001   8.958088  
         2002   4.544406  
         2003   6.169982  
         2004   6.152274  
         2005   6.263276  
         2006   6.098292  
         2007   6.703338  
         2008   4.560118  
         2009   4.051449  
         2010   4.226219  
         2011   2.826410  
         2012   1.587602  
         2013   1.185969  
         2014   1.985308  
...                  ...  
Zimbabwe 2001   0.259089  
         2002  -9.873262  
         2003 -17.871331  
         2004  -6.866223  
         2005  -6.895310  
         2006  -4.816677  
         2007  -5.139678  
         2008 -19.056831  
         2009   9.974501  
         2010  10.377426  
         2011  13.035717  
         2012  12.250240  
         2013   3.120643  
         2014  -0.239734  
         2015  -0.667923  

[2080 rows x 6 columns]